### KoBERT기반 컬럼예측 CTA를 위한 코드입니다

학습용 텍스트 데이터 만들기
- 각 셀의 값을 text로 저장하고, 해당 셀의 컬럼명을 label로 저장

전처리
- 결측치 제거, 특수문자 이상치 제거
- 길이 너무 짧은 텍스트 제거 (ex. 1글자 이하)
- 필요시 strip(), lower() 등 텍스트 정리

사용 이유
-  "성명", "출생", "성별", "지역"과 같은 정형적 개념을 구분해야 하므로
→ 사전학습된 언어모델인 KoBERT를 기반으로 → 텍스트 → 의미(Label) 분류 문제로 설계했습니다.

학습 방식
- 전체 구조는 KoBERT + Linear Layer + Softmax + CrossEntropy Loss 입니다
즉, [CLS] 토큰에서 나온 문장 임베딩을 받아
→ 선형 분류기로 필요한만큼 클래스로 매핑합니다.

- batch_size, learning_rate, epoch, scheduler 등은 실험적으로 조정하며 학습이 과적합되지 않도록 조절합니다.

예측방식
- 학습된 모델은 문장을 입력받아 [1, num_classes] 형태의 logits를 출력합니다.
이 logits에 softmax를 적용해 클래스별 확률을 얻고, argmax()로 예측 라벨을 뽑거나 특정 라벨(성명, 지역)일 확률만 추출할 수 있습니다.

실험결과해석
- 성명 학습 데이터가 충분했기 때문에,
"성명" 컬럼을 구별하는 정확도가 매우 높게 나왔습니다.
(예: 이름_별명 컬럼 → 성명일 확률 평균 0.999)
- 동일 구조로 "지역", "출생" 등도 학습데이터만 확보되면
STI 전체 자동 분류기로 확장이 가능합니다.



장점
- 기존 Rule-based 방식과 달리 문장 내 의미를 자연스럽게 파악할 수 있음
- 학습된 분류기를 통해 다양한 테이블에도 일반화 가능
- 새로운 개념을 추가하고 싶을 때는 라벨만 확장하면 됨
- 예측값을 확률로 받아 신뢰도 기반 필터링도 가능

결론
- 본 모델은 텍스트 기반 컬럼 의미 분류기로 동작하며,
사전학습된 KoBERT 모델의 표현력과 간단한 Linear Classifier 구조를 결합하여
STI(semantic table interpretation) 작업을 효율적으로 자동화할 수 있습니다.

In [ ]:
!pip install transformers
!pip install torch

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/operations/check.py", line 101, in check_install_conflicts
    package_set, _ = create_package_set_from_installed()
              

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### 파인튜닝용 training 코드

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from transformers import BertTokenizer, BertModel, get_scheduler

# 1. 데이터 불러오기
df = pd.read_csv("train_mixed_name.csv", encoding='utf-8-sig')
label_map = {label: i for i, label in enumerate(df["label"].unique())}
df["label_id"] = df["label"].map(label_map)

# 2. 토크나이저 및 BERT 모델 로드
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
bertmodel = BertModel.from_pretrained('monologg/kobert')

# 3. 커스텀 Dataset 정의
class NameDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoded = tokenizer(self.texts[idx], padding='max_length', truncation=True, max_length=32, return_tensors="pt")
        return {
            'input_ids': encoded['input_ids'].squeeze(),
            'attention_mask': encoded['attention_mask'].squeeze(),
            'label': torch.tensor(self.labels[idx])
        }

dataset = NameDataset(df["text"].tolist(), df["label_id"].tolist())
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# 4. 분류기 모델 정의
class BERTClassifier(nn.Module):
    def __init__(self, bert, hidden_size=768, num_classes=4):
        super().__init__()
        self.bert = bert
        self.classifier = nn.Linear(hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output  # [CLS] 토큰 임베딩
        return self.classifier(pooled_output)

# 5. 학습 준비
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERTClassifier(bertmodel, num_classes=len(label_map)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
loss_fn = nn.CrossEntropyLoss()

# 6. Scheduler 설정
num_epochs = 7
num_training_steps = len(dataloader) * num_epochs
scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# 7. 학습 루프
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids, attention_mask)
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"✅ Epoch {epoch+1} 완료 - 평균 Loss: {avg_loss:.4f}")

# 8. 모델 저장
torch.save(model.state_dict(), "kobert_name_finetuned.pt")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


✅ Epoch 1 완료 - 평균 Loss: 0.8601
✅ Epoch 2 완료 - 평균 Loss: 0.5106
✅ Epoch 3 완료 - 평균 Loss: 0.3266
✅ Epoch 4 완료 - 평균 Loss: 0.3110
✅ Epoch 5 완료 - 평균 Loss: 0.3059
✅ Epoch 6 완료 - 평균 Loss: 0.3646
✅ Epoch 7 완료 - 평균 Loss: 0.3579


### 컬럼 예측

In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from torch import nn
import torch.nn.functional as F

# 훈련 시 사용한 라벨 맵 고정
label_map = {"성명": 0, "출생": 1, "성별": 2, "지역": 3}

# 모델 정의
class BERTClassifier(nn.Module):
    def __init__(self, bert, hidden_size=768, num_classes=4):
        super().__init__()
        self.bert = bert
        self.classifier = nn.Linear(hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        return self.classifier(pooled_output)

# 모델 및 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained("monologg/kobert")
bertmodel = BertModel.from_pretrained("monologg/kobert")
model = BERTClassifier(bertmodel, num_classes=4)
model.load_state_dict(torch.load("kobert_name_finetuned.pt", map_location="cpu"))
model.eval()

# 성명 예측 함수
def predict_is_name(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=32)
    with torch.no_grad():
        logits = model(inputs['input_ids'], inputs['attention_mask'])
        probs = torch.softmax(logits, dim=1)
        return probs[0][label_map["성명"]].item()

# 판결문 CSV 불러오기
df = pd.read_csv("judgement.csv", encoding='cp949')

# 컬럼별 평균 성명확률 계산
results = []
for col in df.columns:
    values = df[col].dropna().astype(str).tolist()[:100]
    probs = [predict_is_name(val) for val in values]
    avg_prob = sum(probs) / len(probs) if probs else 0
    results.append((col, avg_prob))

# 출력
results.sort(key=lambda x: x[1], reverse=True)
print("🧠 CSV에서 '성명' 컬럼으로 가장 유력한 후보:")
for col, score in results:
    print(f" {col}: 성명일 확률 평균 {score:.4f}")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


🧠 CSV에서 '성명' 컬럼으로 가장 유력한 후보:
 이름_별명: 성명일 확률 평균 0.9995
 주문: 성명일 확률 평균 0.1504
 생산년도: 성명일 확률 평균 0.0008
 마이크로필름번호: 성명일 확률 평균 0.0008
 판결날짜: 성명일 확률 평균 0.0008
 연번: 성명일 확률 평균 0.0008
 본적주소: 성명일 확률 평균 0.0008
 판결문_번역본_제공: 성명일 확률 평균 0.0008
 판결문_원문_제공: 성명일 확률 평균 0.0008
 관리번호: 성명일 확률 평균 0.0008
 당시나이: 성명일 확률 평균 0.0008
 사건개요: 성명일 확률 평균 0.0008
 죄명: 성명일 확률 평균 0.0008


In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from torch import nn
import torch.nn.functional as F

# 훈련 시 사용한 라벨 맵 고정
label_map = {"성명": 0, "출생": 1, "성별": 2, "주소": 3}

# 모델 정의
class BERTClassifier(nn.Module):
    def __init__(self, bert, hidden_size=768, num_classes=4):
        super().__init__()
        self.bert = bert
        self.classifier = nn.Linear(hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        return self.classifier(pooled_output)

# 모델 및 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained("monologg/kobert")
bertmodel = BertModel.from_pretrained("monologg/kobert")
model = BERTClassifier(bertmodel, num_classes=4)
model.load_state_dict(torch.load("kobert_name_finetuned.pt", map_location="cpu"))
model.eval()

# 성명 예측 함수
def predict_is_name(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=32)
    with torch.no_grad():
        logits = model(inputs['input_ids'], inputs['attention_mask'])
        probs = torch.softmax(logits, dim=1)
        return probs[0][label_map["성명"]].item()

# 판결문 CSV 불러오기
df = pd.read_csv("judgement2.csv", encoding='cp949')

# 컬럼별 평균 성명확률 계산
results = []
for col in df.columns:
    values = df[col].dropna().astype(str).tolist()[:100]
    probs = [predict_is_name(val) for val in values]
    avg_prob = sum(probs) / len(probs) if probs else 0
    results.append((col, avg_prob))

# 출력
results.sort(key=lambda x: x[1], reverse=True)
print("🧠 CSV에서 '성명' 컬럼으로 가장 유력한 후보:")
for col, score in results:
    print(f" {col}: 성명일 확률 평균 {score:.4f}")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


🧠 CSV에서 '성명' 컬럼으로 가장 유력한 후보:
 이름_이름: 성명일 확률 평균 0.9995
 주문: 성명일 확률 평균 0.1504
 생산년도: 성명일 확률 평균 0.0008
 마이크로필름번호: 성명일 확률 평균 0.0008
 판결날짜: 성명일 확률 평균 0.0008
 연번: 성명일 확률 평균 0.0008
 본적주소: 성명일 확률 평균 0.0008
 판결문_번역본_제공: 성명일 확률 평균 0.0008
 판결문_원문_제공: 성명일 확률 평균 0.0008
 관리번호: 성명일 확률 평균 0.0008
 당시나이: 성명일 확률 평균 0.0008
 사건개요: 성명일 확률 평균 0.0008
 죄명: 성명일 확률 평균 0.0008
